# Migrating Workloads to Claude 3.7

This notebook demonstrates how to effectively migrate existing prompts and workflows from Claude 3.5 Sonnet to Claude 3.7, with a focus on leveraging the new extended thinking capability. You'll learn:

1. How to identify prompts that would benefit from extended thinking
2. Techniques for refactoring chain-of-thought prompts
3. Best practices for prompt simplification
4. Before/after examples with performance comparisons

## Prerequisites
- Understanding of Claude 3.5 Sonnet's capabilities
- Familiarity with chain-of-thought prompting
- Knowledge of Python and the Bedrock API
- Completion of previous lessons (especially Lessons 1-2 on extended thinking)

Let's begin by setting up our environment and importing the necessary libraries.

In [ ]:
# Import required libraries
import boto3
import json
import time
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML

# Import our utility functions from previous lessons
import claude_utils

# Set up the Bedrock clients using our utility module
REGION = 'us-west-2'  # Change to your preferred region
bedrock, bedrock_runtime = claude_utils.create_bedrock_clients(REGION)

# Define model IDs for comparison
CLAUDE_35_SONNET_MODEL_ID = 'us.anthropic.claude-3-5-sonnet-20241022-v2:0'
CLAUDE_37_SONNET_MODEL_ID = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'

# Verify model availability
claude_utils.verify_model_availability(bedrock, CLAUDE_37_SONNET_MODEL_ID)
claude_utils.verify_model_availability(bedrock, CLAUDE_35_SONNET_MODEL_ID)

## Understanding When to Migrate

Before diving into specific migration techniques, it's important to understand which prompts are good candidates for migration to Claude 3.7's extended thinking capability.

### Good Candidates for Migration:
- Prompts that use explicit chain-of-thought instructions
- Complex reasoning tasks requiring >1024 tokens of thinking
- Multi-step problem solving workflows
- Tasks that benefit from methodical analysis

### Less Suitable for Migration:
- Simple factual queries
- Basic content generation
- Tasks that don't require detailed reasoning
- Workflows that already work well with standard prompting

Let's examine some concrete examples to understand these differences.

In [ ]:
def compare_models(prompt, chain_of_thought=True, reasoning_budget=4096):
    """
    Compare responses between Claude 3.5 with chain-of-thought and Claude 3.7 with extended thinking
    
    Args:
        prompt (str): The base prompt to test
        chain_of_thought (bool): Whether to add chain-of-thought instructions for 3.5
        reasoning_budget (int): Token budget for 3.7's extended thinking
        
    Returns:
        tuple: (Claude 3.5 response, Claude 3.7 response)
    """
    # Create 3.5 prompt with optional chain-of-thought
    if chain_of_thought:
        cot_prompt = f"""
        Let's solve this step by step:
        1. First, carefully analyze what's being asked
        2. Break down the problem into parts
        3. Solve each part methodically
        4. Combine the results for a final answer

        {prompt}
        """
    else:
        cot_prompt = prompt
    
    # Get 3.5 response
    response_35 = claude_utils.invoke_claude(
        bedrock_runtime,
        cot_prompt,
        CLAUDE_35_SONNET_MODEL_ID,
        enable_reasoning=False,
        max_tokens=1000
    )
    
    # Get 3.7 response with extended thinking
    response_37 = claude_utils.invoke_claude(
        bedrock_runtime,
        prompt,  # Note: No CoT instructions needed
        CLAUDE_37_SONNET_MODEL_ID,
        enable_reasoning=True,
        reasoning_budget=reasoning_budget,
        max_tokens=1000
    )
    
    return response_35, response_37

## Example 1: Mathematical Problem Solving

Let's start with a classic example: a mathematical word problem that traditionally used chain-of-thought prompting. This type of problem is an excellent candidate for migration because it:
- Requires structured reasoning
- Benefits from showing intermediate steps
- Often involves multiple calculations
- Needs validation of intermediate results

We'll compare how this problem was handled in Claude 3.5 versus how it can be solved more elegantly with Claude 3.7's extended thinking.

In [ ]:
# Traditional math word problem
math_problem = """
A store is having a 30% off sale. A customer buys three items:
- A jacket originally priced at $120
- A pair of shoes originally priced at $85
- A shirt originally priced at $45

If there is an additional 10% discount for spending over $200 (calculated after the 30% sale discount),
how much does the customer save in total, and what is their final cost?

Show all calculations clearly.
"""

# Compare the models' responses
response_35, response_37 = compare_models(
    math_problem,
    chain_of_thought=True,  # Enable CoT for 3.5
    reasoning_budget=2048   # Moderate budget for this problem
)

# Display responses for comparison
print("Claude 3.5 Response (with chain-of-thought):")
print("-" * 80)
claude_utils.display_claude_response(response_35)

print("\nClaude 3.7 Response (with extended thinking):")
print("-" * 80)
claude_utils.display_claude_response(response_37)

### Key Differences in the Responses

Let's analyze how the two models approached this problem:

1. **Prompt Structure**
   - Claude 3.5 needed explicit step-by-step instructions
   - Claude 3.7 developed its own reasoning approach

2. **Response Quality**
   - Completeness of calculations
   - Clarity of explanation
   - Validation of results

3. **Token Efficiency**
   - Compare prompt lengths
   - Compare response lengths
   - Overall token usage

These differences highlight why extended thinking can be more effective than traditional chain-of-thought prompting for mathematical reasoning tasks.

In [ ]:
def display_comparison_metrics(response_35, response_37):
    """
    Display detailed comparison metrics between the two responses
    """
    # Calculate metrics for 3.5
    tokens_35 = response_35.get('usage', {}).get('totalTokens', 0)
    time_35 = response_35.get('_elapsed_time', 0)
    
    # Calculate metrics for 3.7
    tokens_37 = response_37.get('usage', {}).get('totalTokens', 0)
    time_37 = response_37.get('_elapsed_time', 0)
    
    # Display comparison
    print("Performance Comparison:")
    print("-" * 40)
    print(f"Claude 3.5 (with CoT):")
    print(f"- Total tokens: {tokens_35:,}")
    print(f"- Response time: {time_35:.2f} seconds")
    print(f"- Tokens per second: {tokens_35/time_35:.1f}")
    print("\nClaude 3.7 (with extended thinking):")
    print(f"- Total tokens: {tokens_37:,}")
    print(f"- Response time: {time_37:.2f} seconds")
    print(f"- Tokens per second: {tokens_37/time_37:.1f}")
    
    # Calculate efficiency gains/losses
    token_diff = ((tokens_37 - tokens_35) / tokens_35) * 100
    time_diff = ((time_37 - time_35) / time_35) * 100
    
    print("\nEfficiency Comparison:")
    print(f"Token usage change: {token_diff:+.1f}%")
    print(f"Response time change: {time_diff:+.1f}%")

# Display metrics for our math problem example
display_comparison_metrics(response_35, response_37)

## Example 2: Analysis and Recommendations

For our second example, let's look at a more complex analytical task that requires synthesizing information and making recommendations. This type of prompt commonly used chain-of-thought to:
- Structure the analysis process
- Ensure consideration of multiple factors
- Guide the development of recommendations
- Maintain logical flow

This example will demonstrate how Claude 3.7's extended thinking can handle complex analysis more naturally than prescribed chain-of-thought steps.

In [ ]:
# Complex analysis problem
analysis_prompt = """
Analyze the potential impact of implementing a four-day work week at a software company with 500 employees.

Consider:
- Employee productivity and satisfaction
- Project timelines and deadlines
- Customer support availability
- Operating costs and profitability
- Team coordination and meetings
- Industry competitiveness

Provide specific recommendations for successfully implementing this change.
"""

# Compare the models' responses
response_35, response_37 = compare_models(
    analysis_prompt,
    chain_of_thought=True,     # Enable CoT for 3.5
    reasoning_budget=4096      # Larger budget for complex analysis
)

# Display responses for comparison
print("Claude 3.5 Response (with chain-of-thought):")
print("-" * 80)
claude_utils.display_claude_response(response_35)

print("\nClaude 3.7 Response (with extended thinking):")
print("-" * 80)
claude_utils.display_claude_response(response_37)

### Migration Pattern: From Structured to Natural Reasoning

This example reveals a key pattern in migrating from Claude 3.5 to 3.7:

1. **Traditional Chain-of-Thought Pattern**
   - Required explicit analysis steps
   - Needed structural guidance
   - Often resulted in formulaic responses

2. **Extended Thinking Pattern**
   - Develops natural reasoning flow
   - Integrates multiple perspectives organically
   - Produces more nuanced analysis

### Key Benefits of Migration
- More comprehensive analysis
- Better integration of related factors
- More natural flow of ideas
- Stronger, more contextual recommendations

This pattern is particularly valuable for complex analytical tasks where rigid structure might limit insight development.

In [ ]:
def analyze_response_patterns(response_35, response_37):
    """
    Analyze response patterns between Claude 3.5 and 3.7
    
    Args:
        response_35 (dict): Response from Claude 3.5
        response_37 (dict): Response from Claude 3.7
        
    Returns:
        None: Prints analysis to stdout
    """
    # Safely extract responses with error handling
    text_35 = claude_utils.extract_response_content(response_35) or ""
    text_37 = claude_utils.extract_response_content(response_37) or ""
    
    if not text_35 or not text_37:
        print("Error: One or both responses are empty")
        return
    
    try:
        # Analyze structure
        sections_35 = len([l for l in text_35.split('\n') if l.strip().startswith('#')])
        sections_37 = len([l for l in text_37.split('\n') if l.strip().startswith('#')])
        
        bullets_35 = len([l for l in text_35.split('\n') if l.strip().startswith('-')])
        bullets_37 = len([l for l in text_37.split('\n') if l.strip().startswith('-')])
        
        # Calculate paragraphs safely
        paragraphs_35 = len([p for p in text_35.split('\n\n') if p.strip()])
        paragraphs_37 = len([p for p in text_37.split('\n\n') if p.strip()])
        
        # Calculate words safely
        words_35 = len(text_35.split())
        words_37 = len(text_37.split())
        
        # Display analysis
        print("Response Pattern Analysis:")
        print("-" * 40)
        print("Claude 3.5 Structure:")
        print(f"- Main sections: {sections_35}")
        print(f"- Bullet points: {bullets_35}")
        print(f"- Average words per paragraph: {words_35/max(1, paragraphs_35):.1f}")
        print(f"- Claude 3.5 v2 Response: \n\n{text_35}")
        
        print("\nClaude 3.7 Structure:")
        print(f"- Main sections: {sections_37}")
        print(f"- Bullet points: {bullets_37}")
        print(f"- Average words per paragraph: {words_37/max(1, paragraphs_37):.1f}")
        print(f"- Claude 3.7 Response: \n\n{text_37}")
        
        # Analyze differences in approach
        print("\nKey Differences:")
        print(f"- Section organization: {'More' if sections_37 > sections_35 else 'Less'} structured")
        print(f"- Point presentation: {'More' if bullets_37 > bullets_35 else 'Less'} bullet points")
        print(f"- Writing style: {'More' if words_37 > words_35 else 'Less'} detailed")
        
    except Exception as e:
        print(f"Error analyzing responses: {str(e)}")

# Analyze patterns in our analysis example
analyze_response_patterns(response_35, response_37)

## Best Practices for Prompt Migration

When migrating prompts from Claude 3.5 to Claude 3.7, several key practices help ensure successful transitions:

### 1. Remove Unnecessary Structure
- Eliminate explicit step-by-step instructions
- Remove artificial thinking markers
- Let extended thinking develop natural flow

### 2. Adjust Token Budgets
- Start with minimum 1024 tokens for extended thinking
- Scale budget based on task complexity
- Monitor and optimize based on results

### 3. Focus on Clear Requirements
- State objectives clearly
- Specify constraints directly
- Provide relevant context

Let's examine these practices with a systematic example.

#### Prompt Migration Helper:

In [ ]:
def demonstrate_prompt_migration(original_prompt, verbose=True):
    """
    Use Claude 3.7 to migrate a prompt from chain-of-thought style to extended thinking style
    
    Args:
        original_prompt (str): Original chain-of-thought style prompt
        verbose (bool): Whether to show detailed transformation steps
    
    Returns:
        tuple: (original prompt, migrated prompt)
    """
    # Create a prompt for Claude to handle the migration
    migration_request = f"""
    Help me migrate this prompt from a chain-of-thought style (used with Claude 3.5) to a cleaner style for Claude 3.7's extended thinking.

    Original prompt:
    {original_prompt}

    Guidelines for migration:
    - Remove explicit step-by-step instructions
    - Remove artificial thinking markers
    - Remove any model-specific steering language for example, instructions about laziness or verbosity
    - Remove chain-of-thought guidance and logic
    - Preserve important context and requirements
    - Make the prompt more natural and direct
    - Keep the core question/request clear
    
    Please provide only the migrated prompt with no additional explanation.
    """
    
    # Get Claude's migration suggestion
    response = claude_utils.invoke_claude(
        bedrock_runtime,
        migration_request,
        CLAUDE_37_SONNET_MODEL_ID,
        enable_reasoning=True,
        reasoning_budget=2048,  # Moderate budget for this task
        max_tokens=1000
    )
    
    # Extract the migrated prompt
    migrated_prompt = claude_utils.extract_response_content(response)
    
    if verbose:
        print("Original Prompt:")
        print("-" * 40)
        print(original_prompt)
        print("\nMigrated Prompt:")
        print("-" * 40)
        print(migrated_prompt)
        
    return original_prompt, migrated_prompt

In [ ]:
# Example of a typical chain-of-thought prompt
original_prompt = """
Let's solve this complex optimization problem step by step:

You are planning a conference with the following constraints:
1. First, determine the venue capacity needed for 300 attendees
2. Then, calculate the budget including:
   - Venue rental ($5000/day)
   - Catering ($75/person/day)
   - Equipment rental ($2000/day)
3. Next, optimize the schedule for:
   - 3 parallel tracks
   - 45-minute sessions
   - 15-minute breaks
4. Finally, recommend the optimal conference duration in days

Think carefully about each step before providing recommendations.
"""

# Demonstrate migration
orig, migrated = demonstrate_prompt_migration(original_prompt)

# Test both versions
print("\nTesting both versions:")
response_35, response_37 = compare_models(
    migrated,  # Use migrated prompt for both to compare approaches
    chain_of_thought=True,  # 3.5 gets CoT instructions
    reasoning_budget=4096   # 3.7 gets extended thinking
)

# Show results
print("\nResults Comparison:")
display_comparison_metrics(response_35, response_37)

## Systematic Migration Patterns

Based on our examples, we can identify several common patterns when migrating prompts from Claude 3.5 to Claude 3.7:

### Pattern 1: Step-by-Step to Goal-Oriented
- **Before**: Detailed step sequence with explicit instructions
- **After**: Clear goal statement with relevant constraints and context

### Pattern 2: Explicit to Implicit Reasoning
- **Before**: "Think through X, Y, Z considerations"
- **After**: "Consider the following factors: X, Y, Z"

### Pattern 3: Structured Output to Natural Output
- **Before**: "Format your response with sections A, B, C"
- **After**: "Include information about A, B, and C in your response"

Let's see these patterns in practice with more examples.

## Check-in Point: Prompt Migration Principles

Let's review what we've learned about migrating prompts:

1. **Structural Changes**
   - What types of instructional markers can be safely removed?
   - How does prompt length change after migration?
   - What elements should be preserved?

2. **Performance Impact**
   - How does token usage compare?
   - What are the response time differences?
   - Does response quality improve?

3. **Best Practices**
   - When should you retain some structure?
   - How do you choose appropriate reasoning budgets?
   - What makes a prompt migration successful?

These insights will help guide your own prompt migration efforts.

In [ ]:
# Define examples of different migration patterns
migration_examples = {
    "step_by_step": {
        "before": """
        Let's solve this math problem step by step:
        1. First, calculate the total cost before discount
        2. Then, apply the 15% discount
        3. Next, add 8% sales tax
        4. Finally, determine the final price
        
        How much would a $120 item cost after discount and tax?
        """,
        "complexity": "simple"
    },
    
    "explicit_reasoning": {
        "before": """
        Think through the implications of remote work by considering:
        First, analyze productivity impacts.
        Then, examine employee satisfaction effects.
        Next, evaluate communication challenges.
        Finally, weigh cost savings against potential drawbacks.
        
        Is remote work a net positive for most organizations?
        """,
        "complexity": "medium"
    },
    
    "structured_output": {
        "before": """
        Analyze this short story following these steps:
        1. First, summarize the plot
        2. Then, identify the main themes
        3. Next, analyze the character development
        4. Finally, discuss the author's writing style
        
        Structure your analysis with clear sections for each aspect.
        """,
        "complexity": "complex"
    }
}

# Migrate and display each example
for pattern_name, example in migration_examples.items():
    print(f"\n\n{'='*80}\nMigration Pattern: {pattern_name.replace('_', ' ').title()}")
    print(f"Complexity: {example['complexity'].title()}")
    
    # Migrate the prompt
    _, migrated = demonstrate_prompt_migration(example["before"], verbose=False)
    
    # Display before/after
    print("\nBefore:")
    print("-" * 40)
    print(example["before"])
    
    print("\nAfter:")
    print("-" * 40)
    print(migrated)
    
    # Determine appropriate budget based on complexity
    budget_map = {"simple": 1024, "medium": 2048, "complex": 4096}
    budget = budget_map.get(example["complexity"], 2048)
    
    print(f"\nRecommended reasoning budget: {budget} tokens")

## Practical Migration Workflow

For systematically migrating your existing prompts to Claude 3.7, follow this practical workflow:

### 1. Inventory and Prioritize
- List all your existing prompts
- Identify those with explicit chain-of-thought instructions
- Prioritize complex reasoning tasks that would benefit most

### 2. Analyze Current Prompt
- Identify reasoning instructions
- Note required context and constraints
- Recognize output format requirements

### 3. Simplify and Migrate
- Remove explicit reasoning steps
- Preserve critical context and requirements
- Consider using Claude 3.7 itself to help with migration

### 4. Test and Refine
- Start with minimum reasoning budget (1024 tokens)
- Increase budget if needed based on task complexity
- Compare results and adjust as needed

This systematic approach ensures successful migration while optimizing for Claude 3.7's capabilities.

In [ ]:
def create_migration_checklist(prompt):
    """
    Generate a migration checklist for a given prompt
    
    Args:
        prompt (str): The original prompt to analyze
        
    Returns:
        dict: Migration recommendations
    """
    analysis_request = f"""
    Analyze this prompt that was designed for Claude 3.5 with chain-of-thought instructions.
    
    Prompt:
    {prompt}
    
    Please create a migration checklist with the following information in JSON format:
    1. Elements to remove (e.g., step-by-step instructions, thinking markers)
    2. Elements to preserve (e.g., key context, important constraints)
    3. Recommended reasoning budget (1024, 2048, 4096, or 8192 tokens)
    4. Complexity assessment (simple, medium, complex, very complex)
    
    Respond with only the JSON data.
    """
    
    # Get Claude's analysis
    response = claude_utils.invoke_claude(
        bedrock_runtime,
        analysis_request,
        CLAUDE_37_SONNET_MODEL_ID,
        enable_reasoning=True,
        reasoning_budget=2048,
        max_tokens=1000
    )
    
    # Extract and parse JSON
    checklist_text = claude_utils.extract_response_content(response)
    
    try:
        # Try to parse as JSON
        import json
        checklist = json.loads(checklist_text)
        
        # Display in a readable format
        print("Migration Checklist:")
        print("-" * 40)
        print("Elements to Remove:")
        for item in checklist.get("Elements to remove", []):
            print(f"- {item}")
            
        print("\nElements to Preserve:")
        for item in checklist.get("Elements to preserve", []):
            print(f"- {item}")
            
        print(f"\nComplexity Assessment: {checklist.get('Complexity assessment', 'Unknown')}")
        print(f"Recommended Reasoning Budget: {checklist.get('Recommended reasoning budget', 1024)} tokens")
        
        return checklist
    
    except json.JSONDecodeError:
        print("Error parsing response as JSON. Raw response:")
        print(checklist_text)
        return None

# Test the migration checklist function
complex_prompt = """
Let's analyze this company's financial performance step by step:

1. First, calculate the year-over-year revenue growth rate
2. Then, analyze the profit margins (gross, operating, net)
3. Next, evaluate the debt-to-equity ratio and liquidity metrics
4. Then, compare performance against industry benchmarks
5. Finally, identify key strengths and areas for improvement

Provide a structured analysis with clear sections for each aspect.
"""

migration_checklist = create_migration_checklist(complex_prompt)

## Handling Complex Migration Scenarios

While many prompts can be easily migrated following our patterns, certain scenarios require special consideration:

### Multi-Stage Reasoning Workflows

For complex workflows that previously relied on a sequence of reasoning steps, the migration strategy needs to:
- Preserve critical dependencies between steps
- Maintain clarity about the overall goal
- Ensure all necessary context is included

### Domain-Specific Requirements

Some domains (like mathematical proofs, scientific analysis, or code generation) may have specific expectations about:
- Notation and formatting
- Methodology and approach
- Verification and validation

Let's examine how to handle these more complex migrations.

In [ ]:
# Example of a multi-stage reasoning workflow
multistage_prompt = """
Let's solve this complex data analysis problem step by step:

1. First, analyze the customer segmentation data:
   - Identify primary customer segments
   - Calculate average revenue per segment
   - Determine growth rate for each segment

2. Then, evaluate marketing channel effectiveness:
   - Compare customer acquisition cost by channel
   - Calculate ROI for each marketing channel
   - Identify the most and least effective channels

3. Next, forecast future performance:
   - Project segment growth for next 12 months
   - Estimate marketing budget requirements
   - Predict overall revenue impact

4. Finally, prioritize recommendations:
   - Rank segments by potential value
   - Suggest budget reallocation between channels
   - Propose specific strategies for high-value segments

Ensure your analysis follows this structure and provides clear recommendations.
"""

print("Complex Multi-Stage Workflow Example:")
print("-" * 60)

# Migrate the prompt
_, migrated_multistage = demonstrate_prompt_migration(multistage_prompt, verbose=False)

# Show migration results
print("\nOriginal Multi-Stage Prompt:")
print("-" * 40)
print(multistage_prompt)

print("\nMigrated Multi-Stage Prompt:")
print("-" * 40)
print(migrated_multistage)

# Generate and display migration checklist
print("\nMigration Analysis:")
migration_checklist = create_migration_checklist(multistage_prompt)

## Before and After: A Complete Picture

Let's summarize what we've learned by comparing a complete workflow before and after migration:

### Claude 3.5 Approach:
1. Detailed chain-of-thought instructions
2. Explicit step sequencing
3. Structured reasoning guidance
4. Visible thinking process in prompt

### Claude 3.7 Approach:
1. Clear goal statement
2. Relevant context and constraints
3. Extended thinking (handled internally)
4. Appropriate reasoning budget

The key shift is from prescriptive reasoning (telling Claude exactly how to think) to goal-oriented reasoning (telling Claude what you want to achieve and letting its extended thinking handle the process).

Let's visualize this difference with a side-by-side comparison of a complete workflow.
![Complete Workflow](./images/lesson8/compare.png)

## Performance Benchmarking Before and After Migration

To truly validate the benefits of migrating to Claude 3.7's extended thinking, we should systematically benchmark performance. Let's create a framework for comparing:

1. **Reasoning Quality**: How thorough and accurate is the reasoning?
2. **Response Efficiency**: How efficiently are responses generated?
3. **Token Usage**: How does total token consumption compare?

By benchmarking these metrics across different types of prompts, we can make data-driven migration decisions and understand the real-world impact of the migration.

In [ ]:
def benchmark_migration(prompts, reasoning_budgets=None):
    """
    Benchmark performance before and after migration
    
    Args:
        prompts (dict): Dictionary of prompts to benchmark
        reasoning_budgets (dict, optional): Dictionary mapping prompt names to reasoning budgets
        
    Returns:
        pd.DataFrame: Benchmark results
    """
    # Default reasoning budgets if not provided
    if reasoning_budgets is None:
        reasoning_budgets = {
            name: 4096 for name in prompts.keys()
        }
    
    results = []
    
    for name, original_prompt in prompts.items():
        print(f"\nBenchmarking: {name}")
        
        # 1. Migrate the prompt
        _, migrated_prompt = demonstrate_prompt_migration(original_prompt, verbose=False)
        
        # 2. Test Claude 3.5 with original prompt
        start_time = time.time()
        response_35 = claude_utils.invoke_claude(
            bedrock_runtime,
            original_prompt,
            CLAUDE_35_SONNET_MODEL_ID,
            enable_reasoning=False,
            max_tokens=1500
        )
        time_35 = time.time() - start_time
        
        # 3. Test Claude 3.7 with migrated prompt
        start_time = time.time()
        response_37 = claude_utils.invoke_claude(
            bedrock_runtime,
            migrated_prompt,
            CLAUDE_37_SONNET_MODEL_ID,
            enable_reasoning=True,
            reasoning_budget=reasoning_budgets.get(name, 4096),
            max_tokens=1500
        )
        time_37 = time.time() - start_time
        
        # 4. Calculate metrics
        tokens_35 = response_35.get('usage', {}).get('totalTokens', 0)
        tokens_37 = response_37.get('usage', {}).get('totalTokens', 0)
        
        # 5. Extract response text to measure length
        text_35 = claude_utils.extract_response_content(response_35) or ""
        text_37 = claude_utils.extract_response_content(response_37) or ""
        
        # 6. Store results
        results.append({
            'Prompt': name,
            'Original Length': len(original_prompt),
            'Migrated Length': len(migrated_prompt),
            'Prompt Change %': round(((len(migrated_prompt) - len(original_prompt)) / len(original_prompt) * 100), 1),
            'Claude 3.5 Tokens': tokens_35,
            'Claude 3.7 Tokens': tokens_37,
            'Token Change %': round(((tokens_37 - tokens_35) / tokens_35 * 100), 1),
            'Claude 3.5 Time (s)': round(time_35, 2),
            'Claude 3.7 Time (s)': round(time_37, 2),
            'Time Change %': round(((time_37 - time_35) / time_35 * 100), 1),
            'Claude 3.5 Output Words': len(text_35.split()),
            'Claude 3.7 Output Words': len(text_37.split()),
            'Output Change %': round(((len(text_37.split()) - len(text_35.split())) / max(1, len(text_35.split())) * 100), 1),
            'Reasoning Budget': reasoning_budgets.get(name, 4096)
        })
        
        print(f"  Original prompt: {len(original_prompt)} chars")
        print(f"  Migrated prompt: {len(migrated_prompt)} chars")
        print(f"  Claude 3.5: {tokens_35} tokens in {time_35:.2f}s")
        print(f"  Claude 3.7: {tokens_37} tokens in {time_37:.2f}s")
    
    # Create DataFrame
    df = pd.DataFrame(results)
    return df

# Define prompts for benchmarking
benchmark_prompts = {
    "Math Problem": """
    Let's solve this probability problem step by step:
    1. First, understand what we're looking for
    2. Then, identify the probability of each event
    3. Next, apply the appropriate probability formula
    4. Finally, calculate the answer
    
    In a standard deck of 52 cards, what is the probability of drawing a face card (J, Q, K) or an ace?
    """,
    
    "Business Analysis": """
    Let's analyze this business case step by step:
    1. First, identify the key issues
    2. Then, analyze the market conditions
    3. Next, evaluate the financial implications
    4. Finally, recommend a course of action
    
    A retail company is considering expanding to online sales. They have $500K to invest and want to know if they should build their own platform or use an existing marketplace like Amazon.
    """,
    
    "Code Review": """
    Let's review this code step by step:
    1. First, understand what the code is trying to do
    2. Then, check for logic errors
    3. Next, identify performance issues
    4. Finally, suggest improvements
    
    ```python
    def find_duplicates(arr):
        duplicates = []
        for i in range(len(arr)):
            for j in range(i+1, len(arr)):
                if arr[i] == arr[j] and arr[i] not in duplicates:
                    duplicates.append(arr[i])
        return duplicates
    ```
    """
}

# Define reasoning budgets for each prompt
reasoning_budgets = {
    "Math Problem": 2048,
    "Business Analysis": 4096,
    "Code Review": 4096
}

# Run the benchmark
benchmark_df = benchmark_migration(benchmark_prompts, reasoning_budgets)

# Display the results
display(HTML("<h3>Migration Benchmark Results</h3>"))
display(benchmark_df[['Prompt', 'Prompt Change %', 'Token Change %', 'Time Change %', 'Output Change %', 'Reasoning Budget']])

# Create visualizations
try:
    import matplotlib.pyplot as plt
    
    # Plot token and time changes
    fig, ax = plt.subplots(1, 2, figsize=(14, 6))
    
    # Token changes
    benchmark_df.plot(
        x='Prompt', 
        y=['Claude 3.5 Tokens', 'Claude 3.7 Tokens'], 
        kind='bar', 
        ax=ax[0],
        color=['skyblue', 'lightgreen']
    )
    ax[0].set_title('Token Usage Comparison')
    ax[0].set_ylabel('Tokens Used')
    ax[0].tick_params(axis='x', rotation=45)
    
    # Time changes
    benchmark_df.plot(
        x='Prompt', 
        y=['Claude 3.5 Time (s)', 'Claude 3.7 Time (s)'], 
        kind='bar', 
        ax=ax[1],
        color=['skyblue', 'lightgreen']
    )
    ax[1].set_title('Response Time Comparison')
    ax[1].set_ylabel('Time (seconds)')
    ax[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Visualization could not be created: {e}")
    print("Would display charts comparing token usage and response time")

## When to Retain Chain-of-Thought Elements

While we've focused on removing chain-of-thought elements when migrating to Claude 3.7, there are some situations where retaining certain aspects of structured prompting remains beneficial:

### 1. Specific Output Format Requirements
If you require a very specific output format, it can be helpful to specify this clearly. However, focus on the *what* (output format) rather than the *how* (thinking process).

### 2. Domain-Specific Methodologies
In specialized fields with established methodologies (scientific research, legal analysis, etc.), noting the required methodology can be valuable.

### 3. Multiple Distinct Tasks
When your prompt contains several unrelated tasks, structuring these as separate items can help organize the response (though not the reasoning process).

### 4. Very Small Reasoning Budgets
If you're using the minimum reasoning budget (1024 tokens) for a complex task, providing some high-level structure might help.

The key principle is to guide *what* Claude should accomplish without prescribing *how* it should think through the problem - let extended thinking handle the reasoning process.

## Conclusion: Migration Strategy Checklist

We've explored the process of migrating prompts from Claude 3.5 to Claude 3.7, focusing on leveraging the extended thinking capability for improved performance. Here's a checklist to guide your own migration efforts:

1. **Audit Existing Prompts**
   - Identify prompts with explicit reasoning instructions
   - Prioritize complex reasoning tasks
   - Note current token usage and performance

2. **Simplify Prompts**
   - Remove step-by-step instructions
   - Eliminate explicit thinking markers
   - Preserve essential context and requirements

3. **Configure Extended Thinking**
   - Start with minimum reasoning budget (1024 tokens)
   - Scale budget based on task complexity
   - Monitor response quality and adjust as needed

4. **Validate Results**
   - Compare response quality before and after
   - Measure token usage and response times
   - Make incremental improvements based on feedback

### Key Takeaways

- Extended thinking enables more natural, flexible reasoning
- Simpler prompts often lead to better results with Claude 3.7
- Select reasoning budgets based on task complexity
- Focus on what you want (goals) rather than how to get there (steps)

By following these guidelines, you can successfully migrate your existing workloads to take full advantage of Claude 3.7's advanced capabilities.